In [1]:
#Bayesian Optimization using Gaussian Processes for Hyperparameter Tuning with noisy experimental data.

import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.transforms import Standardize, Normalize

%matplotlib inline
%load_ext autoreload
%autoreload 2

torch.manual_seed(0)

def objective_function(X, pHopt =7 , temp_opt =35, a = 100, b = 1, c = 1, noise_level = 0.1):
    """
    Simulates a 2D Gaussian-like response surface with controllable noise.

    Parameters:
    - X: input tensor of shape [n, 2], columns are [pH, temp]
    - pHopt, temp_opt: optimal pH and temperature
    - a: peak value (must be high enough to keep output positive)
    - b, c: curvature coefficients (bigger = narrower peak)
    - noise_level: fraction of y to scale the noise (e.g., 0.1 = 10%)

    Returns:
    - y: simulated noisy response values
    """
    pH, temp = X[:, 0], X[:, 1]
    y = a-b*(pH-pHopt)**2 - c*(temp-temp_opt)**2
    noise = noise_level * y * torch.randn_like(y)
    return y + noise

model = SingleTaskGP(X, y, input_transform = Normalize(d=2), outcome_transform = Standardize(m=1))
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)








class GPWithNoise:
    def __init__(self, X, Y, kernel, noise_variance=0.1, y_norm='meanstd', opt_params=None):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.y_norm = y_norm
        self.opt_params = opt_params if opt_params else {}
        
        self.Y_mean = Y.mean() if y_norm == 'meanstd' else 0
        self.Y_std = Y.std() if y_norm == 'meanstd' else 1
        
        self.model = self._create_gp_model(X, Y, kernel, noise_variance)
        self.mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        
    def _create_gp_model(self, X, Y, kernel, noise_variance):
        Y_norm = (Y - self.Y_mean) / self.Y_std if self.y_norm == 'meanstd' else Y
        likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
        
        # Set initial value of noise (learnable parameter)
        likelihood.noise = torch.tensor(noise_variance, dtype=torch.float32, requires_grad=True)
        
        model = SingleTaskGP(X, Y_norm.unsqueeze(-1), likelihood=likelihood)
        return model.to(self.device)
    
    def optimize(self, steps=100):
        # Create the Adam optimizer
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.1)
        
        for _ in range(steps):
            optimizer.zero_grad()
            
            # Forward pass
            output = self.model(self.model.train_inputs[0])
            loss = -self.mll(output, self.model.train_targets)
            
            # Backward pass
            loss.backward()
            optimizer.step()
    
    def get_acquisition_function(self, acq_type, best_f=None):
        if acq_type == 'EI':
            return ExpectedImprovement(self.model, best_f=best_f)
        elif acq_type == 'UCB':
            return UpperConfidenceBound(self.model, beta=2.0)
        else:
            raise ValueError("Unsupported acquisition function")
    
    def predict(self, X_new):
        self.model.eval()
        with torch.no_grad():
            return self.model(X_new)

# Example usage:
# X and Y are your training data (torch tensors)
# You might have a custom kernel defined already
X_train = torch.rand(10, 1)  # Example training inputs
Y_train = torch.sin(X_train) + 0.1 * torch.randn(10, 1)  # Noisy observations (y = sin(x) + noise)
Y_train = Y_train.squeeze()

# Define a simple kernel (RBF, or any custom kernel)
kernel = gpytorch.kernels.RBFKernel()

gp_model = GPWithNoise(X_train, Y_train, kernel)

# Perform optimization
gp_model.optimize(steps=100)

# Example prediction (new data)
X_new = torch.rand(5, 1)
predictions = gp_model.predict(X_new)


c:\Users\uqkmuroi\gitcode\Torch-Practice\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'X' is not defined